In [1]:
from typing import Tuple

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("dataset.csv")

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,other,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,0,29,1,0,other,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,0,2,1,1,other,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,secondary,0,1506,1,0,other,5,may,92,1,-1,0,unknown,no
4,33,other,single,secondary,0,1,0,0,other,5,may,198,1,-1,0,unknown,no


In [5]:
def transform(data: str) -> int:
    return 1 if data == "yes" else 0


def prepare_data(
    df: pd.DataFrame,
) -> Tuple[
    pd.DataFrame,
    pd.DataFrame,
    pd.DataFrame,
    pd.DataFrame,
    pd.DataFrame,
    pd.DataFrame,
    ColumnTransformer,
]:
    """
    Prepare the bank marketing dataset for training and evaluation.

    The input DataFrame contains demographic, financial, and campaign-related
    information about clients contacted during a direct marketing campaign.
    The target column `y` indicates whether the client subscribed to a term
    deposit (`yes` or `no`).

    Steps (you MUST follow these steps):
    1. Identify feature columns and the target column `y`.
    2. Separate features (X) and target (y).
    3. Encode the target labels:
       - `yes` → 1
       - `no` → 0
    4. Identify categorical and numerical feature columns.
    5. Use ColumnTransformer with OneHotEncoder to encode all categorical columns:
       - use OneHotEncoder(drop="first", sparse_output=False)
       - Pass numerical features without modification
    6. Fit the preprocessor on the training data.
    7. Split the data in TWO stages (keep stratification):
       - First split into train and test:
            * test_size = 0.2
            * random_state = 42
            * stratify = y
       - Then split the training part into train and validation:
            * test_size = 0.2   (20% of the training set)
            * random_state = 42
            * stratify = y_train
    8. Return:
         X_train, X_val, X_test, y_train, y_val, y_test, preprocessor

    Notes:
    - The returned X arrays must be fully numeric.
    - The returned y arrays must contain binary labels with shape (N, 1) or (N,).
    - The data must be suitable for PyTorch binary classification.
    """
    x = df.drop(columns=["y"])
    y = df["y"].map({"yes": 1, "no": 0}).values.reshape(-1,1)
    
    cat_cols = x.select_dtypes(include=["object"]).columns.tolist()
    num_cols = x.select_dtypes(include=["int", "float"]).columns.tolist()
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(drop="first", sparse_output=False), cat_cols)
        ],
        remainder="passthrough",
    )
    
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
    x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=42,stratify=y_train)
    x_train = preprocessor.fit_transform(x_train)
    x_val = preprocessor.transform(x_val)
    x_test = preprocessor.transform(x_test)
    
    return x_train, x_val, x_test, y_train, y_val, y_test, preprocessor

In [6]:
class BankMarketingDataset(Dataset):
    """
    A PyTorch Dataset for bank marketing binary classification.

    Each sample consists of:
    - a numeric feature vector representing a client's profile and campaign data
    - a binary label indicating whether the client subscribed to a term deposit

    Requirements:
    - __init__(self, X, y):
        * X: numpy array of numeric features
        * y: array-like of binary labels (0 or 1)
        * Store:
            - X as a float32 tensor
            - y as a float32 tensor with shape (N, 1)
    - __len__(self):
        * Return the number of samples
    - __getitem__(self, idx):
        * Return (X[idx], y[idx])
    """
    
    def __init__(self,x,y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1,1)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [7]:
def train_one_epoch(
    model: nn.Module,
    train_loader: DataLoader,
    criterion,
    optimizer,
) -> float:
    """
    Train the model for ONE epoch on the training dataset.

    This is a binary classification task for predicting whether a client
    subscribes to a term deposit.

    Requirements:
    - Set the model to training mode using model.train()
    - Iterate over batches from train_loader
    - For each batch:
        * Compute model outputs (logits or probabilities)
        * Compute the loss using Binary Cross-Entropy loss
          (BCELoss or BCEWithLogitsLoss)
        * Zero the gradients
        * Perform backpropagation
        * Update model parameters using the optimizer
    - Accumulate the training loss over all batches
    - Return the average training loss as a float
      (total loss divided by the number of batches)
    """
    
    model.train()
    losses = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        
    return losses / len(train_loader)

In [8]:
def evaluate(
    model: nn.Module,
    val_loader: DataLoader,
    criterion: nn.Module,
) -> Tuple[float, float]:
    """
    Evaluate the model on the validation dataset.

    This is a binary classification task for bank marketing outcome prediction.

    Requirements:
    - Set the model to evaluation mode using model.eval()
    - Disable gradient computation using torch.no_grad()
    - Iterate over batches from val_loader
    - For each batch:
        * Compute model outputs
        * Compute and accumulate validation loss
        * Convert outputs to predicted labels using threshold 0.5
        * Collect predicted labels and true labels
    - Compute validation accuracy over the entire validation set
    - Return:
        - validation accuracy (float)
        - validation loss (float)
    """
    model.eval()
    losses = 0
    total = 0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses += loss.item()
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return (
        correct / total,
        losses / len(val_loader)
    )

In [9]:
def test(
    model: nn.Module,
    test_loader: DataLoader,
) -> tuple[Tensor, Tensor]:
    """
    Evaluate the trained model on the test dataset.

    This function performs inference for binary classification of
    bank marketing campaign outcomes.

    Requirements:
    - Set the model to evaluation mode using model.eval()
    - Disable gradient computation using torch.no_grad()
    - Iterate over batches from test_loader
    - For each batch:
        * Compute model outputs
        * Convert outputs to predicted labels using threshold 0.5
        * Collect all predicted labels and true labels
    - Return:
        - Tensor of true labels (shape: N,)
        - Tensor of predicted labels (shape: N,)

    These outputs will be used to compute a classification report.
    """
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            all_preds.extend(predicted.tolist())
            all_labels.extend(labels.tolist())
            
            
    return (torch.tensor(all_labels),torch.tensor(all_preds))

In [10]:
def build_model_1(input_dim: int) -> nn.Module:
    """
    Build and return a PyTorch neural network for bank marketing
    binary classification.

    Requirements:
    - Use nn.Sequential to define the model
    - The model must accept input vectors of size input_dim
    - The final layer must output a single value
    - Do NOT apply Sigmoid if using BCEWithLogitsLoss

    Note:
    - Use Binary Cross-Entropy loss during training
    - This model will serve as the baseline architecture
    """
    
    return nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid(),
    )

In [11]:
def build_model_2(input_dim: int) -> nn.Module:
    """
    Build and return a second PyTorch neural network for bank marketing
    binary classification.

    This model should differ from build_model_1
    (e.g. more layers, more neurons, dropout, different activations).

    Requirements:
    - Use nn.Sequential to define the model
    - The model must accept input vectors of size input_dim
    - The final layer must output a single value
    - Do NOT apply Sigmoid if using BCEWithLogitsLoss

    Note:
    - Use Binary Cross-Entropy loss during training
    - This model will be compared against build_model_1
    """

    return nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Dropout(0.33),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid(),
    )

In [12]:
df = pd.read_csv("./dataset.csv")
X_train, X_val, X_test, y_train, y_val, y_test, preprocessor = prepare_data(df)

train_loader = DataLoader(
    BankMarketingDataset(X_train, y_train),
    batch_size=64,
)
val_loader = DataLoader(
    BankMarketingDataset(X_val, y_val),
    batch_size=64,
)
test_loader = DataLoader(
    BankMarketingDataset(X_test, y_test),
    batch_size=64,
)

input_dim = X_train.shape[1]
model_1 = build_model_1(input_dim)
model_2 = build_model_2(input_dim)

criterion_1 = nn.BCELoss()
criterion_2 = nn.BCELoss()

optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=1e-3)
optimizer_2 = torch.optim.Adam(model_2.parameters(), lr=1e-3)

In [13]:
epochs = 25
train_losses_1 = []
val_losses_1 = []
val_accuracies_1 = []

for epoch in range(epochs):
    # Call all required functions and store the computed metrics
    # (training loss, validation loss, and validation accuracy).

    train_loss = train_one_epoch(
        model_1,
        train_loader,
        criterion_1,
        optimizer_1,
    )
    val_acc, val_loss = evaluate(
        model_1,
        val_loader,
        criterion_1,
    )

    train_losses_1.append(train_loss)
    val_losses_1.append(val_loss)
    val_accuracies_1.append(val_acc)

    print(
        f"Epoch {epoch + 1}/{epochs} | Train loss: {train_loss:.4f} | Val acc: {val_acc:.4f}"
    )

Epoch 1/25 | Train loss: 0.8281 | Val acc: 0.8804
Epoch 2/25 | Train loss: 0.4417 | Val acc: 0.8036
Epoch 3/25 | Train loss: 0.4602 | Val acc: 0.8707
Epoch 4/25 | Train loss: 0.3667 | Val acc: 0.8829
Epoch 5/25 | Train loss: 0.3308 | Val acc: 0.8872
Epoch 6/25 | Train loss: 0.3320 | Val acc: 0.8886
Epoch 7/25 | Train loss: 0.3030 | Val acc: 0.8918
Epoch 8/25 | Train loss: 0.2986 | Val acc: 0.8966
Epoch 9/25 | Train loss: 0.2812 | Val acc: 0.8958
Epoch 10/25 | Train loss: 0.2655 | Val acc: 0.8948
Epoch 11/25 | Train loss: 0.2704 | Val acc: 0.8937
Epoch 12/25 | Train loss: 0.2600 | Val acc: 0.8962
Epoch 13/25 | Train loss: 0.2549 | Val acc: 0.8960
Epoch 14/25 | Train loss: 0.2507 | Val acc: 0.8967
Epoch 15/25 | Train loss: 0.2507 | Val acc: 0.8962
Epoch 16/25 | Train loss: 0.2427 | Val acc: 0.8977
Epoch 17/25 | Train loss: 0.2437 | Val acc: 0.8945
Epoch 18/25 | Train loss: 0.2469 | Val acc: 0.8940
Epoch 19/25 | Train loss: 0.2393 | Val acc: 0.8936
Epoch 20/25 | Train loss: 0.2445 | Val a